In [10]:
import boto3
import json
import arxiv
import urllib
import pdfx

In [11]:
search = arxiv.Search(
    query = "computer science &  ai",
    # id_list=["1605.08386v1"]
    max_results = 1,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
    print ("Titre :", result.title)
    print ("Auteurs :", result.authors)
    print ("Lien :", result.pdf_url)

Titre : Temporal Alignment Networks for Long-term Video
Auteurs : [arxiv.Result.Author('Tengda Han'), arxiv.Result.Author('Weidi Xie'), arxiv.Result.Author('Andrew Zisserman')]
Lien : http://arxiv.org/pdf/2204.02968v1


In [12]:
response = urllib.request.urlopen(result.pdf_url)    
file = open("Test" + ".pdf", 'wb')
file.write(response.read())
file.close()

filename = 'Test.pdf'

## Lecture du PDF

### Pour mettre un PDF au choix, décommenter et indiquer le path

In [13]:
# filename = '...'

In [14]:
pdf = pdfx.PDFx(filename)
metadata = pdf.get_metadata()
references_dict = pdf.get_references_as_dict()
text = pdf.get_text()

In [15]:
def after_references(mypdftext): 
    keyword1 = 'References'
    keyword2 = 'REFERENCES'
    keyword3 = 'R EFERENCES'
    keyword4 = 'Reference'
    keyword5='[1]' 

    if keyword1 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword1)
    elif keyword2 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword2)
    elif keyword3 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword3)
    elif keyword4 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword4)
    elif keyword5 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword5)
    else:
        after_keyword = mypdftext[:10000]
    return after_keyword

#All references in a variable

references = after_references(text)

# AWS Comprehend

## Indiquez vos identifiants d'access

In [16]:
ACCESS_ID='...'
ACCESS_KEY='...'
session_token='...'

#### Access Personnel (A laisser en commentaire)

In [17]:
# Access Personnel

# import toml

# access = toml.load('aws_access.toml')

# ACCESS_ID=access['default']['aws_access_key_id']
# ACCESS_KEY=access['default']['aws_secret_access_key']
# session_token=access['default']['aws_session_token']

In [28]:

comprehend = boto3.client(service_name='comprehend', region_name='us-east-1',aws_access_key_id=ACCESS_ID,
                                            aws_secret_access_key= ACCESS_KEY,aws_session_token=session_token
)


results = comprehend.detect_entities(Text=references[:3000], LanguageCode='en')

clean_results = []
i=0
for r in results['Entities']:
    line={}
    if r["Type"] == "PERSON" and r["Score"] > 0.95:
        line['id'] = i
        line['Person']= r['Text']
        clean_results.append(line)
        i += 1

print(json.dumps(clean_results))


[{"id": 0, "Person": "Yuki M. Asano"}, {"id": 1, "Person": "Christian Rupprecht"}, {"id": 2, "Person": "Andrea Vedaldi"}, {"id": 3, "Person": "Max Bain"}, {"id": 4, "Person": "Arsha Nagrani"}, {"id": 5, "Person": "G\u00a8ul Varol"}, {"id": 6, "Person": "Andrew Zisser"}, {"id": 7, "Person": "Subhabrata Bhattacharya"}, {"id": 8, "Person": "Mahdi M. Kalayeh"}, {"id": 9, "Person": "Rahul Suk-\nthankar"}, {"id": 10, "Person": "Mubarak Shah"}, {"id": 11, "Person": "Avrim Blum"}, {"id": 12, "Person": "Tom Mitchell"}, {"id": 13, "Person": "Piotr Bojanowski"}, {"id": 14, "Person": "R\u00b4emi Lajugie"}, {"id": 15, "Person": "Francis Bach"}, {"id": 16, "Person": "Ivan Laptev"}, {"id": 17, "Person": "Jean Ponce"}, {"id": 18, "Person": "Cordelia Schmid"}, {"id": 19, "Person": "Josef Sivic"}, {"id": 20, "Person": "Piotr Bojanowski"}, {"id": 21, "Person": "R\u00b4emi Lajugie"}, {"id": 22, "Person": "Edouard Grave"}, {"id": 23, "Person": "Fran-\ncis Bach"}, {"id": 24, "Person": "Ivan Laptev"}, {"id":